In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import torch
import torch.utils.data
print("PyTorch Version : {}".format(torch.__version__))
import torchtext
print("Torch Text Version : {}".format(torchtext.__version__))
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from torchtext.data import to_map_style_dataset
from torchtext import data
from torchtext import datasets
import torch.nn as nn
import torch.optim as optim

PyTorch Version : 1.13.1+cu117
Torch Text Version : 0.14.0


In [18]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
torch.cuda.is_available()
#cuda是否可用；
#torch.cuda.device_count()
#返回gpu数量；
#torch.cuda.get_device_name(0)
#返回gpu名字，设备索引默认从0开始；
#torch.cuda.current_device()
#返回当前设备索引；
#nvcc --version
#nvidia-smi

True

1.创建分词器

In [19]:
from torchtext.data import get_tokenizer

tokenizer = get_tokenizer("basic_english") ## We'll use tokenizer available from PyTorch

tokenizer("Hello, How are you?")

['hello', ',', 'how', 'are', 'you', '?']

2.导入GloVe词向量

In [20]:

import torchtext.vocab as vocab
from torchtext.vocab import GloVe

# cache_dir是保存golve词典的缓存路径
cache_dir = '.vector_cache/glove.840B.300d'
# dim是embedding的维度
global_vectors = vocab.GloVe(name='6B', dim=300, cache=cache_dir) 

也可以在线下载

In [21]:
'''
from torchtext.vocab import GloVe


global_vectors = GloVe(name='840B', dim=300)
'''

"\nfrom torchtext.vocab import GloVe\n\n\nglobal_vectors = GloVe(name='840B', dim=300)\n"

In [22]:
embeddings = global_vectors.get_vecs_by_tokens(tokenizer("Hello, How are you?"), lower_case_backup=True)

embeddings.shape

torch.Size([6, 300])

In [23]:
global_vectors.get_vecs_by_tokens(["."], lower_case_backup=True)


tensor([[-1.2559e-01,  1.3630e-02,  1.0306e-01, -1.0123e-01,  9.8128e-02,
          1.3627e-01, -1.0721e-01,  2.3697e-01,  3.2870e-01, -1.6785e+00,
          2.2393e-01,  1.2409e-01, -8.6708e-02,  3.3010e-01,  3.4375e-01,
         -8.7582e-04, -2.9658e-01,  2.4417e-01, -1.1592e-01, -3.5742e-02,
         -1.0830e-02,  2.0776e-01,  2.9285e-01, -7.3491e-02, -1.8598e-01,
         -2.0090e-01, -9.5366e-02,  6.3732e-03, -1.3620e-01,  9.2028e-02,
         -3.9957e-02,  1.9027e-01, -1.0456e-01,  2.7670e-03, -7.1742e-01,
         -1.2915e-01, -1.3451e-03,  2.7002e-01, -5.3023e-02,  2.2148e-01,
          1.3881e-01, -1.5051e-01, -1.9150e-01,  1.6402e-01,  9.7484e-02,
          5.6841e-02,  3.9789e-01,  4.0725e-01,  1.4802e-01,  2.1569e-01,
         -1.0671e-01, -1.0232e-01,  2.4810e-02, -2.2100e-01, -1.0720e-02,
          1.4234e-01, -2.8242e-01,  1.9254e-01,  8.6720e-02, -3.8970e-01,
          1.1321e-01,  1.3779e-03,  6.4009e-03, -1.6206e-01, -8.2153e-02,
         -5.5397e-01,  3.6789e-01, -4.

3.加载数据、分词和词向量

In [24]:
from torch.utils.data import DataLoader
from torchtext.data.functional import to_map_style_dataset

max_words = 200
embed_len = 300

def vectorize_batch(batch):
    Y, X = list(zip(*batch))
    X = [tokenizer(x) for x in X]
    X = [tokens+[""] * (max_words-len(tokens))  if len(tokens)<max_words else tokens[:max_words] for tokens in X]
    X_tensor = torch.zeros(len(batch), max_words, embed_len)
    Y_tensor = torch.zeros(len(batch),2)
    for i, tokens in enumerate(X):
        X_tensor[i] = global_vectors.get_vecs_by_tokens(tokens)

    #for i, target in enumerate(Y):
       # if (target==1):
          #  Y_tensor[i] = torch.tensor([1,0])
        #if (target==2):
          #  Y_tensor[i] = torch.tensor([0,1])
    return X_tensor,torch.tensor(Y)-1 #Y_tensor 



train_dataset, test_dataset  = torchtext.datasets.IMDB()

train_dataset, test_dataset = to_map_style_dataset(train_dataset), to_map_style_dataset(test_dataset)


train_loader = DataLoader(train_dataset, batch_size=100, shuffle=True,collate_fn=vectorize_batch)
test_loader  = DataLoader(test_dataset, batch_size=100,shuffle=False, collate_fn=vectorize_batch)

4.定义模型和训练

In [25]:
class BiRNN(nn.Module):
    def __init__(self, features, num_hiddens,
                 num_layers, **kwargs):
        super(BiRNN, self).__init__(**kwargs)
        # 将bidirectional设置为True以获取双向循环神经网络

        self.encoder = nn.LSTM(features, num_hiddens, num_layers=num_layers,batch_first=True,
                                bidirectional=True)
        self.decoder = nn.Linear(4*num_hiddens, 2)
    def forward(self, inputs):
        #self.encoder.flatten_parameters()
        outputs,_= self.encoder(inputs)
        outs = self.decoder( torch.cat((outputs[:,0,:], outputs[:,-1,:]), dim=1))
      
        return  outs#outs

In [26]:
features=300
num_layers=2
num_hiddens=100

In [27]:

model = BiRNN(features, num_hiddens, num_layers).to(device)
print(model)
criterion = nn.CrossEntropyLoss(reduction="none")
lr = 0.001
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

BiRNN(
  (encoder): LSTM(300, 100, num_layers=2, batch_first=True, bidirectional=True)
  (decoder): Linear(in_features=400, out_features=2, bias=True)
)


In [28]:
for epoch in range(10):
    model.train()
    correct = 0
    total = 0
    for x, y in train_loader:
        optimizer.zero_grad()
        x, y = x.to(device), y.to(device)
        pred = model(x)
        _, predictions = torch.max(pred, 1)
        '''
        print(y)
        print(pred)
        print(_)
        print(predictions)
        '''
        total += y.size(0)
        correct += (predictions == y).sum().item()
        loss = criterion(pred, y).requires_grad_(True)
        loss.sum().backward()
        optimizer.step()
    print(f'Accuracy of the network on the train dataset: {100 * correct // total} %')
    train_acc=correct // total
    model.eval()
    correct = 0
    total = 0
    for x, y in test_loader:
        x, y = x.to(device), y.to(device)
        with torch.no_grad():
            pred = model(x)
            _, predictions = torch.max(pred, 1)
    
            total += y.size(0)
            correct += (predictions == y).sum().item()
    test_acc=correct // total
    print(f'Accuracy of the network on the test dataset: {100 * correct // total} %')


    


Accuracy of the network on the train dataset: 64 %
Accuracy of the network on the test dataset: 63 %
Accuracy of the network on the train dataset: 75 %
Accuracy of the network on the test dataset: 75 %
Accuracy of the network on the train dataset: 77 %
Accuracy of the network on the test dataset: 75 %
Accuracy of the network on the train dataset: 82 %
Accuracy of the network on the test dataset: 82 %
Accuracy of the network on the train dataset: 84 %
Accuracy of the network on the test dataset: 83 %
Accuracy of the network on the train dataset: 86 %
Accuracy of the network on the test dataset: 84 %
Accuracy of the network on the train dataset: 87 %
Accuracy of the network on the test dataset: 85 %
Accuracy of the network on the train dataset: 89 %
Accuracy of the network on the test dataset: 85 %
Accuracy of the network on the train dataset: 90 %
Accuracy of the network on the test dataset: 84 %
Accuracy of the network on the train dataset: 92 %
Accuracy of the network on the test data

In [29]:
def predict_sentiment(net,  sequence):
    """预测文本序列的情感"""
    sequence=tokenizer(sequence)
    if len(sequence)<max_words :
        sequence = sequence+[""] * (max_words-len(sequence))   
    else: 
        sequence[:max_words] 
    
    sequence_tensor = torch.zeros(1, max_words, embed_len)
    sequence_tensor = global_vectors.get_vecs_by_tokens(sequence)

    pred=net(sequence_tensor.unsqueeze(0) .to(device))
    _, predictions = torch.max(pred, 1)
    

    return 'positive' if predictions == 1 else 'negative'

In [30]:
predict_sentiment(model, 'this movie is so great')


'positive'

In [31]:
predict_sentiment(model, 'I love you')


'positive'

In [41]:
predict_sentiment(model, 'this movie is so bad')


'negative'